In [148]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences,to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, SimpleRNN, LSTM, GRU, Input, InputLayer, Embedding, TimeDistributed,Bidirectional
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords,brown, treebank
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')
import pickle

In [79]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [80]:
import nltk
nltk.download('brown')


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [81]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [82]:
d1=brown.tagged_sents(tagset='universal')

In [83]:
len(d1)

57340

In [84]:
d2=treebank.tagged_sents(tagset='universal')

In [85]:
len(d2)

3914

In [86]:
data=d1+d2

In [87]:
len(data)

61254

In [113]:
x=[]
y=[]

for i in data:
    x1=[]
    y1=[]
    for j in i:
        x1.append(j[0])
        y1.append(j[1])
    x.append(x1)
    y.append(y1)

In [114]:
len(x)

61254

In [115]:
len(y)

61254

In [116]:
c=0

for i in range(len(x)):
    if(len(x[i])==len(y[i])):
        pass
    else:
        c+=1
        
print(c)
        

0


In [117]:
tk_x=Tokenizer(oov_token='UNK')
tk_x.fit_on_texts(x)

tk_y=Tokenizer(oov_token='UNK')
tk_y.fit_on_texts(y)

In [120]:
len(tk_x.word_index)

53233

In [121]:
len(tk_y.word_index)

13

In [122]:
x_data=tk_x.texts_to_sequences(x)
y_data=tk_y.texts_to_sequences(y)

In [123]:
len(x_data)

61254

In [124]:
final_x=pad_sequences(x_data,padding='post')
final_y=pad_sequences(y_data,padding='post')

In [125]:
final_x.shape

(61254, 271)

In [126]:
final_y.shape

(61254, 271)

In [127]:
final_y=to_categorical(final_y)

In [128]:
final_y.shape

(61254, 271, 14)

In [129]:
#x_train,x_test,y_train,y_test=train_test_split(final_x,final_y,test_size=0.2,random_state=1,stratify=final_y)

In [131]:
model=Sequential()
model.add(InputLayer(shape=(271,)))
model.add(Embedding(53233+1,5))
model.add(Bidirectional(SimpleRNN(units=50,return_sequences=True)))
model.add(TimeDistributed(Dense(units=14, activation='softmax')))

In [132]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ (None, 271, 5)              │         266,170 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 271, 100)            │           5,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 271, 14)             │           1,414 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 273,184 (1.04 MB)

 Trainable params: 273,184 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

In [133]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [134]:
model.fit(final_x,final_y,batch_size=120,epochs=5,validation_split=0.2)

Epoch 1/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 397s 831ms/step - accuracy: 0.9232 - loss: 0.3696 - val_accuracy: 0.9739 - val_loss: 0.0932
Epoch 2/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 354s 858ms/step - accuracy: 0.9812 - loss: 0.0665 - val_accuracy: 0.9881 - val_loss: 0.0486
Epoch 3/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 349s 851ms/step - accuracy: 0.9931 - loss: 0.0274 - val_accuracy: 0.9909 - val_loss: 0.0367
Epoch 4/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 326s 796ms/step - accuracy: 0.9957 - loss: 0.0154 - val_accuracy: 0.9920 - val_loss: 0.0330
Epoch 5/5
409/409 ━━━━━━━━━━━━━━━━━━━━ 340s 830ms/step - accuracy: 0.9970 - loss: 0.0107 - val_accuracy: 0.9926 - val_loss: 0.0311


In [135]:
test=['Hi everyone ,I am Nikhitha']

In [136]:
tk_x.texts_to_sequences(test)

[[12576, 1233, 28, 450, 1]]

In [137]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(test))

['hi everyone i am UNK']

In [139]:
test=pad_sequences(tk_x.texts_to_sequences(test),maxlen=271,padding='post')

In [146]:
np.argmax(model.predict(test)[0],axis=1)[np.argmax(model.predict(test)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


array([7, 2, 9, 3], dtype=int64)

In [147]:
tk_y.sequences_to_texts([[7, 2, 9, 3]])

['adj noun pron verb']

In [150]:
pickle.dump(tk_x,open(r"C:\Users\LENOVO\Desktop\POS tag\tk_x.pkl",'wb'))